# EDA, Balancing and Sampling

In [ ]:
import os.path

import pandas as pd
from pandas_profiling import ProfileReport

In [ ]:
generate_reports: bool = True
generate_dfs: bool = True

In [ ]:
part_1 = pd.read_csv("dataset/dataset-part1.csv")
part_2 = pd.read_csv("dataset/dataset-part2.csv")

In [ ]:
df = pd.concat([part_1, part_2], ignore_index=True)

## Exploratory Data Analysis

In [ ]:
df.columns.values

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe(include=['object'])

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns].head(5)

In [ ]:
df["LABEL"].unique()

In [ ]:
# Check for nan columns
df.columns[df.isna().any()].tolist()
# No nan

In [ ]:
if not os.path.isfile("EDA.html") and generate_reports:
    profile = ProfileReport(df, title="Exploratory Data Analysis - Clean DF", minimal=True)
    profile.to_file("EDA.html")

## Datset Balancing

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler(sampling_strategy='minority')

In [ ]:
labels_dic = {
    0: "Normal flow",
    1: "SYN Scan - aggressive",
    2: "Denial of Service R-U-Dead-Yet",
    3: "Denial of Service Slowloris"
}

In [ ]:
normal_flow = df.loc[df["LABEL"] == labels_dic[0]]
syn_attacks = df.loc[df["LABEL"] == labels_dic[1]]
r_u_dead_attacks = df.loc[df["LABEL"] == labels_dic[2]]
dos_attacks = df.loc[df["LABEL"] == labels_dic[3]]

In [ ]:
print(len(normal_flow))
print(len(syn_attacks))
print(len(r_u_dead_attacks))
print(len(dos_attacks))

### Oversampling DDoS attack

In [ ]:
attacks_oversampling = pd.concat([syn_attacks, dos_attacks])
y_oversampling = attacks_oversampling["LABEL"]
x_oversampling = attacks_oversampling.drop(["LABEL"], axis= 1)

In [ ]:
X_oversampling_res, y_oversampling_res = ros.fit_resample(x_oversampling, y_oversampling)

In [ ]:
print('Resampled dataset shape %s' % Counter(y_oversampling_res))

In [ ]:
oversampling_result = X_oversampling_res.copy()
oversampling_result["LABEL"] = y_oversampling_res.copy()

In [ ]:
df = pd.concat([normal_flow, r_u_dead_attacks, oversampling_result])

### Undersampling benign

from imblearn.under_sampling import RandomUnderSampler

In [ ]:
normal_flow = df.loc[df["LABEL"] == labels_dic[0]]
syn_attacks = df.loc[df["LABEL"] == labels_dic[1]]
r_u_dead_attacks = df.loc[df["LABEL"] == labels_dic[2]]
dos_attacks = df.loc[df["LABEL"] == labels_dic[3]]

In [ ]:
benign_us = pd.concat([syn_attacks, normal_flow])

In [ ]:
y = benign_us["LABEL"]
X = benign_us.drop(["LABEL"], axis=1)

In [ ]:
rus = RandomUnderSampler(random_state=0)

In [ ]:
X_undersampled, y_undersampled = rus.fit_resample(X, y)

print('Resampled dataset shape %s' % Counter(y_undersampled))

In [ ]:
X_undersampled["LABEL"] = y_undersampled
undersample_res = X_undersampled

In [ ]:
df = pd.concat([dos_attacks, r_u_dead_attacks, undersample_res])

### Generate balanced csv file

In [ ]:
# create csv if it doesn't exist
if not os.path.isfile("dataset/balanced_df.csv") and generate_dfs:
    df.to_csv("dataset/balanced_df.csv", index_label=False)
else:
    df =pd.read_csv("dataset/balanced_df.csv")


df.shape

## Sampling

In [ ]:
if not os.path.isfile("dataset/sampled_df.csv") and generate_dfs:
    df = df.groupby('LABEL', group_keys=False).apply(lambda x: x.sample(frac=0.05))
    df.to_csv("dataset/sampled_df.csv", index_label=False)
else:
    df = pd.read_csv("dataset/sampled_df.csv")

df.shape